<a href="https://colab.research.google.com/github/mitramir55/Kaggle_NLP_competition/blob/master/Error_in_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip3 install tensorflow-hub
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 1.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30315 sha256=2cb2085af40f97a1cbca2f98653f6315f441dd8f28b17adf8a60324586b57e22
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=e7d51cee6ea6ae07d4090e16af3aa737cafa9c3600edefdd43da938bd05c5ca4
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=ff01a267c86c5ea43d43fc53a8e56e47cb7d1a97cdf4f1d2eec2951c9df2134c
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 2.8MB/s 


In [2]:
#!pip install tensorflow-hub
#!pip install tf-nightly
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.3.0
Hub version:  0.8.0


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re
import string

import bert
from tensorflow.keras import layers
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model 
from keras.preprocessing.sequence import pad_sequences
import math
import spacy
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [4]:
!git clone https://github.com/mitramir55/Kaggle_NLP_competition.git
train = pd.read_csv('Kaggle_NLP_competition/train.csv')
test = pd.read_csv('Kaggle_NLP_competition/test.csv')
df_concat = pd.read_csv('Kaggle_NLP_competition/df_concat_cleaned.csv')
test_id = test.id

Cloning into 'Kaggle_NLP_competition'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 60 (delta 24), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [5]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2", 
                            trainable = True)
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()

tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

print('Bert Tokenizer is ready!!!')

max_len = 0
serie = df_concat.cleaned_text.str.split(' ')
for tweet in serie:
    x = len(tweet)
    if x > max_len:
        max_len = x
        
del serie
max_len += 2
print(max_len)


Bert Tokenizer is ready!!!
29


In [6]:
train_cleaned = df_concat[:train.shape[0]]
test_cleaned = df_concat[train.shape[0]:]

In [7]:
class DisasterDetector:
    
    def __init__(self, tokenizer, bert_layer, max_len =30, lr = 0.0001,
                 epochs = 15, batch_size = 32, dtype = tf.int32 ,
                 activation = 'sigmoid', optimizer = 'SGD',
                 beta_1=0.9, beta_2=0.999, epsilon=1e-07,
                 metrics = 'accuracy', loss = 'binary_crossentropy'):
        
        self.lr = lr
        self.epochs = epochs
        self.max_len = max_len
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.bert_layer = bert_layer
        self.models = []

        self.activation = activation
        self.optimizer = optimizer
        self.dtype = dtype
        
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon =epsilon
        
        self.metrics = metrics
        self.loss = loss
        
    def encode(self, texts):
        all_tokens = []
        masks = []
        segments = []
        
        for text in texts:
            
            tokenized = self.tokenizer.convert_tokens_to_ids(['[CLS]'] + self.tokenizer.tokenize(text) + ['[SEP]'])
            
            len_zeros = self.max_len - len(tokenized)
            
            
            padded = tokenized + [0] * len_zeros
            mask = [1] * len(tokenized) + [0] * len_zeros
            segment = [0] * self.max_len
            
            all_tokens.append(padded)
            masks.append(mask)
            segments.append(segment)
            
        print(len(all_tokens[0]))
        return np.array(all_tokens), np.array(masks), np.array(segments)
        
    def make_model(self):
        

        input_word_ids = Input(shape = (self.max_len, ), dtype=tf.int32,
                            name = 'input_word_ids')
        
        input_mask = Input(shape = (self.max_len, ), dtype=tf.int32,
                           name = 'input_mask')
        
        segment_ids = Input(shape = (self.max_len, ), dtype=tf.int32,
                            name = 'segment_ids')


        #pooled output is the output of dimention and

        pooled_output, sequence_output = self.bert_layer([input_word_ids,
                                                     input_mask,
                                                     segment_ids])

        clf_output = sequence_output[:, 0, :]
        out = tf.keras.layers.Dense(1, activation = self.activation)(clf_output)
        #out = tf.keras.layers.Dense(1, activation = 'sigmoid', input_shape =  (clf_output,) )(clf_output)
        

        model = Model(inputs = [input_word_ids, input_mask, segment_ids],
                      outputs = out)
        if self.optimizer is 'SGD':
            optimizer = SGD(learning_rate = self.lr)

        elif self.optimizer is 'Adam': 
            optimizer = Adam(learning_rate = self.lr, beta_1=self.beta_1,
                             beta_2=self.beta_2, epsilon=self.epsilon)

        model.compile(loss = self.loss, optimizer = self.optimizer,
                      metrics = [self.metrics])
        
        return model
    
    
    
    
    def train(self, x, k = 3):    
        kfold = StratifiedKFold(n_splits = k, shuffle = True)


        for fold, (train_idx, val_idx) in enumerate(kfold.split(x['cleaned_text'], x['target'])):
            print('fold: ', fold)

            x_trn = self.encode(x.loc[train_idx, 'cleaned_text'])
            x_val = self.encode(x.loc[val_idx, 'cleaned_text'])
            y_trn = np.array(x.loc[train_idx, 'target'], dtype = np.uint8)
            y_val = np.array(x.loc[val_idx, 'target'], dtype = np.uint8)
            print('the data type of y train: ', type(y_trn))
            print('x_val shape', x_val[0].shape)
            print('x_trn shape', x_trn[0].shape)
            
            model = self.make_model()
            print('model made.')
            model.fit(x_trn, y_trn,
                    validation_data = (x_val, y_val),
                    batch_size=self.batch_size, epochs = self.epochs)

            self.models.append(model)

    def predict(self, x):
        y_preds = []
        X_test_encoded = self.encode(x['cleaned_text'])
        y_pred = np.zeros((X_test_encoded[0].shape[0], 1))

        for model in self.models:
            y_pred = model.predict(X_test_encoded)
            y_preds.append(y_pred)
        return y_preds

        

In [8]:


classifier = DisasterDetector(tokenizer = tokenizer, bert_layer = bert_layer, max_len = max_len, lr = 0.0001,
                  epochs = 10,  activation = 'sigmoid',
                batch_size = 32,optimizer = 'SGD',
                beta_1=0.9, beta_2=0.999, epsilon=1e-07)


In [9]:
classifier.train(train_cleaned)

fold:  0
29
29
the data type of y train:  <class 'numpy.ndarray'>
x_val shape (2538,)
x_trn shape (5075,)
model made.


ValueError: ignored